### 라이브러리 설치

In [3]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 데이터 읽기

In [4]:
(x_train, y_train),(x_test,y_test)=cifar10.load_data()

In [7]:
###데이터 정규화
x_train=x_train.astype(np.float32)/255
x_test=x_test.astype(np.float32)/255

In [9]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 1) (10000, 1)


### 데이터 증강 (150% 증대)

In [11]:
gen = ImageDataGenerator(rotation_range=20, #회전
                         shear_range=0.2, #기울기
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         horizontal_flip=True)

In [13]:
# 보강할 학습데이터 이미지 생성
augment_ratio = 1.5   # 전체 데이터의 150%
augment_size = int(augment_ratio * x_train.shape[0])
print(augment_size)

75000


In [15]:
# 전체 x_train 개수의 150% 비율만큼
randidx = np.random.randint(x_train.shape[0], size=augment_size)

np.random.randint는 랜덤한 인덱스를 생성하는 함수로 특정 범위 내에서 정수를 임의로 선택하는 함수입니다.
-x_train.shape[0]은 원본 데이터셋의 갯수를 나타냅니다.
-np.random.randint(50000,size=75000)

0부터 50,000사이의 랜덤한 정수를 75000개를 생성하고,
이렇게 생성한 랜덤한 숫자들은 x_train과 y_train에서 무작위로 데이터를 선택하기 위한 인덱스입니다.

In [17]:
# 임의로 선택된 데이터는 원본데이터를 참조하기 때문에 원본데이터에 영향을 줄수 있음. 
#그래서 copy() 함수를 통해 안전하게 복사본 만듬
x_augmented = x_train[randidx].copy()  
y_augmented = y_train[randidx].copy()
print(x_augmented.shape, y_augmented.shape)
print(type(x_augmented))

(75000, 32, 32, 3) (75000, 1)
<class 'numpy.ndarray'>


In [19]:
#  이미지 보강 실행
x_augmented, y_augmented = gen.flow(x_augmented, 
                                    y_augmented,batch_size=augment_size, 
                                    shuffle=False).__next__()
print(x_augmented.shape, y_augmented.shape)

(75000, 32, 32, 3) (75000, 1)


In [20]:
# x_train, y_train 에 보강된 데이터 추가
x_train = np.concatenate( (x_train, x_augmented) )
y_train = np.concatenate( (y_train, y_augmented) )

print(x_train.shape, y_train.shape)

(125000, 32, 32, 3) (125000, 1)


In [ ]:
# CNN 모델 구축
cnn=Sequential()

In [ ]:
cnn.add(Conv2D(32,(3,3),activation='relu', padding='same',input_shape=(32,32,3))) #높이,너비,채널
cnn.add(Conv2D(32,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(64,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten()) #3차원 텐서를 1차원 벡터로 변환

cnn.add(Dense(128,activation='relu')) #은닉층 개념
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax')) #출력층 

cnn.summary()

In [ ]:
#컴퓨터 서버 스펙이 부족해서...250 epoch 돌리는게 어렵네요 ;;
cnn.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=256,epochs=250,validation_data=(x_test,y_test))

In [ ]:
cnn.evaluate(x_test,y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy Trend')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='best')
plt.grid()
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss Trend')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='best')
plt.grid()
plt.show()